# CatBoost  
In this notebook, I implement a CatBoost model based on outfit tags encoding.  

This approach failed because it returned a score of 0 for all evaluation metrics.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path='/content/drive/MyDrive/RecSys_206894495'
%run /content/drive/MyDrive/RecSys_206894495/models/evaluate_models.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
empty json


In [ ]:
!pip install pyarrow
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [ ]:
file_path_outfit = path + "/models/val_outfit.json"
file_path_group = path + "/models/val_group.json"

In [ ]:
#import
#warning
import warnings
warnings.filterwarnings('ignore')

#general
import pandas as pd
import numpy as np
from itertools import product
import itertools
import json
from tqdm.notebook import tqdm
from datetime import datetime

import gc

#for predictions
from catboost import CatBoostClassifier, Pool, metrics
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
#load data
user_splits=pd.read_parquet(path+'/models/user_splits.parquet')

In [ ]:
# Function to create a new DataFrame for a specific set (train, val, test)
def create_new_df(df, set_type):
    new_data = []

    for index, row in df.iterrows():
        outfit_ids = row[f'{set_type}_outfit_ids']
        unique_outfit_ids, counts = np.unique(outfit_ids, return_counts=True)
        for outfit_id, count in zip(unique_outfit_ids, counts):
            new_data.append({
                'customer_id': row['customer_id'],
                'outfit_id': outfit_id,
                'label': 1
            })

    return pd.DataFrame(new_data)

# Create new DataFrames for train, val, and test
train_df = create_new_df(user_splits, 'train')
val_df = create_new_df(user_splits, 'val')
test_df = create_new_df(user_splits, 'test')


In [ ]:
# Function to create a new DataFrame for a specific set (train, val, test)
def create_new_df_group(df, set_type):
    new_data = []

    for index, row in df.iterrows():
        outfit_ids = row[f'{set_type}_group']
        unique_outfit_ids, counts = np.unique(outfit_ids, return_counts=True)
        for outfit_id, count in zip(unique_outfit_ids, counts):
            new_data.append({
                'customer_id': row['customer_id'],
                'group': outfit_id,
                'label': 1
            })

    return pd.DataFrame(new_data)

# Create new DataFrames for train, val, and test
train_df_group = create_new_df_group(user_splits, 'train')
val_df_group = create_new_df_group(user_splits, 'val')
test_df_group = create_new_df_group(user_splits, 'test')

In [ ]:
orders=pd.read_parquet(path+'/archive/data/orders.parquet')
groups=pd.read_parquet(path+'/archive/data/outfits.parquet')[['id','group']]
orders=orders.merge(groups, left_on='outfit.id', right_on='id', how='left').drop('id', axis=1)
del groups

In [ ]:
def generate_balanced_dataset(df,all_combinations, customer_col='customer_id', outfit_col='outfit_id'):

    # Identify positive examples
    positive_examples = df[[customer_col, outfit_col]]

    # Get negative examples by removing positive examples from all combinations
    negative_examples = pd.merge(all_combinations, positive_examples, on=[customer_col, outfit_col], how='left', indicator=True)
    negative_examples = negative_examples[negative_examples['_merge'] == 'left_only'].drop(columns=['_merge'])

    # Sample negative examples to match the size of positive examples
    negative_examples = negative_examples.sample(n=len(positive_examples), random_state=42)

    # Add a label column to distinguish between positive and negative examples
    positive_examples['label'] = 1
    negative_examples['label'] = 0

    # Combine positive and negative examples
    final_df = pd.concat([positive_examples, negative_examples], ignore_index=True)

    return final_df

In [ ]:
# Get unique customer_ids and outfit_ids
customer_ids = orders['customer.id'].unique()
outfit_ids = orders['outfit.id'].unique()
# Generate all possible combinations of customer_id and outfit_id
all_combinations_outfits = pd.DataFrame(list(itertools.product(customer_ids, outfit_ids)), columns=['customer_id', 'outfit_id'])

In [ ]:
# Generate balanced dataset
train_df = generate_balanced_dataset(train_df,all_combinations_outfits)
val_df = generate_balanced_dataset(val_df,all_combinations_outfits)
test_df = generate_balanced_dataset(test_df,all_combinations_outfits)
del all_combinations_outfits

In [ ]:
# Get unique customer_ids and group
customer_ids = orders['customer.id'].unique()
groups = orders['group'].unique()
# Generate all possible combinations of customer_id and outfit_id
all_combinations_group = pd.DataFrame(list(itertools.product(customer_ids, groups)), columns=['customer_id', 'group'])

In [ ]:
# Generate balanced dataset

train_df_group = generate_balanced_dataset(train_df_group,all_combinations_group,outfit_col='group')
val_df_group = generate_balanced_dataset(val_df_group,all_combinations_group,outfit_col='group')
test_df_group = generate_balanced_dataset(test_df_group,all_combinations_group,outfit_col='group')
del all_combinations_group

In [ ]:
train_df.sample(5)

,customer_id,outfit_id,label
71908,4213,outfit.73e8b4069ac14d19a432a0d01a439243,0
13288,2200,outfit.f21c3560fb00439baa401a3fe1315dd3,1
90552,3365,outfit.90153bbf4f8947598358f70f57087821,0
72132,343,outfit.063a311819a74d52b52ecfeba92958ad,0
81668,2773,outfit.518ef36ef26c4c1a9f9de153da7c840e,0


In [ ]:
# Concatenate the two DataFrames
combined_df = pd.concat([train_df, val_df])

# Group by customer_id and outfit_id and sum the counts
join_df = combined_df.groupby(['customer_id', 'outfit_id'], as_index=False)['label'].max()

del combined_df

# Concatenate the two DataFrames
combined_df_group = pd.concat([train_df_group, val_df_group])

# Group by customer_id and outfit_id and sum the counts
join_df_group = combined_df_group.groupby(['customer_id', 'group'], as_index=False)['label'].max()

del combined_df_group

In [ ]:
join_df.sample(5)

,customer_id,outfit_id,label
82625,5733,outfit.c5d0ed62000241acb220236b8c01dc44,1
86042,5994,outfit.7a013c1ffdf04252a7d15c116044ccba,0
86932,6068,outfit.0e313845ef6b41feaa57ad3b0126a099,1
46566,3229,outfit.18e3eb423a9143a9b99d3933e5943246,0
46479,3225,outfit.d3799f343a384143bb38b5f7a2affe0e,0


In [ ]:
outfits=pd.read_parquet(path+'/archive/data/outfits.parquet')

In [ ]:
#prepare tags
all_tags = outfits["outfit_tags"].values.tolist()
mlb = MultiLabelBinarizer()
one_hot_encoded = mlb.fit_transform(all_tags)
outfits_encoded=pd.concat([outfits[['id','group']], pd.DataFrame(one_hot_encoded)], axis=1)
del all_tags, one_hot_encoded, outfits

In [ ]:
groups=outfits_encoded.groupby('group').max().drop('id', axis=1).reset_index()
outfits_encoded.drop('group', axis=1, inplace=True)

In [ ]:
train_df=train_df.merge(outfits_encoded, left_on='outfit_id', right_on='id').drop('id', axis=1)
val_df=val_df.merge(outfits_encoded, left_on='outfit_id', right_on='id').drop('id', axis=1)
test_df=test_df.merge(outfits_encoded, left_on='outfit_id', right_on='id').drop('id', axis=1)
join_df=join_df.merge(outfits_encoded, left_on='outfit_id', right_on='id').drop('id', axis=1)

In [ ]:
train_df.columns

Index(['customer_id',   'outfit_id',       'label',             0,
                   1,             2,             3,             4,
                   5,             6,             7,             8,
                   9,            10,            11,            12,
                  13,            14,            15,            16,
                  17,            18,            19,            20,
                  21,            22,            23,            24,
                  25,            26,            27,            28,
                  29,            30,            31,            32,
                  33,            34,            35,            36,
                  37,            38,            39,            40,
                  41,            42,            43,            44,
                  45,            46,            47,            48,
                  49,            50,            51,            52,
                  53,            54,            55,           

In [ ]:
train_df_group=train_df_group.merge(groups, on='group')
val_df_group=val_df_group.merge(groups, on='group')
test_df_group=test_df_group.merge(groups, on='group')
join_df_group=join_df_group.merge(groups, on='group')

In [ ]:
del outfits_encoded, groups

In [ ]:
x_train=train_df.drop(['label'],axis=1)
y_train=train_df['label']
x_val=val_df.drop(['label'],axis=1)
y_val=val_df['label']
x_test=test_df.drop(['label'],axis=1)
y_test=test_df['label']
x_join=join_df.drop(['label'],axis=1)
y_join=join_df['label']

x_train_group=train_df_group.drop(['label'],axis=1)
y_train_group=train_df_group['label']
x_val_group=val_df_group.drop(['label'],axis=1)
y_val_group=val_df_group['label']
x_test_group=test_df_group.drop(['label'],axis=1)
y_test_group=test_df_group['label']
x_join_group=join_df_group.drop(['label'],axis=1)
y_join_group=join_df_group['label']

In [ ]:
categorical_features_indices_outfit = ['outfit_id','customer_id']
categorical_features_indices_group = ['group','customer_id']

In [ ]:
gc.collect()

0

##CatBoost

In [ ]:
def train_catb(x,y,categorical_features_indices,iterations,depth,learning_rate):
  model = CatBoostClassifier(iterations=iterations,depth=depth,learning_rate=learning_rate)
  model.fit(
    x, y,
    cat_features=categorical_features_indices
  )
  return model
def test_catb(algo,x_test,train,group=False):
  x_test['label_predicted']=algo.predict(x_test)
  results=x_test[x_test['label_predicted']==1]
  df_merged = user_splits_df.merge(results, on='customer_id', how='left').groupby('customer_id').agg({
      **{col: 'first' for col in user_splits_df.columns if col != 'customer_id'},
      'label_predicted': lambda x: list(x)
  }).reset_index()
  # Rename the aggregated column to id_prediction
  if group:
    df_merged.rename(columns={'label_predicted': 'group_prediction'}, inplace=True)
  else:
    df_merged.rename(columns={'label_predicted': 'id_prediction'}, inplace=True)
  return x_test,algo,df_merged

def train_and_test_gb(x_train,y_train,categorical_features_indices,x_test,user_splits_df,iterations,depth,learning_rate,group=False):
  algo=train_catb(x_train,y_train,categorical_features_indices,iterations,depth,learning_rate)
  test,algo,df_merged=test_catb(algo,x_test,user_splits_df,group)
  return test,algo,df_merged

In [ ]:
iterations_list=[2,3,4]
depth_list=[2,3,4]
learning_rate_list=[1,0.5,0.25]

In [ ]:
##Hyper-Parameters tuning for outfit

test_permutations = list(product(iterations_list,depth_list,learning_rate_list))

for iterations,depth,learning_rate in tqdm(test_permutations):
    user_splits_df=user_splits.copy()
    predictions,algo,user_splits_df = train_and_test_gb(x_train,y_train,categorical_features_indices_outfit,x_val,user_splits_df,iterations,depth,learning_rate)
    param_dict = {'iterations':iterations,'depth':depth,'learning_rate':learning_rate}
    user_splits_df, all_dict = evaluate_val_metrics_at_n_outfit(user_splits_df,'CatBoost_tag', n=10,model_params=param_dict)

  0%|          | 0/27 [00:00<?, ?it/s]

0:	learn: 0.4453251	total: 106ms	remaining: 106ms
1:	learn: 0.4219567	total: 155ms	remaining: 0us
0:	learn: 0.4975164	total: 52.5ms	remaining: 52.5ms
1:	learn: 0.4483326	total: 96.4ms	remaining: 0us
0:	learn: 0.5741878	total: 49.2ms	remaining: 49.2ms
1:	learn: 0.5098936	total: 92.5ms	remaining: 0us
0:	learn: 0.4362768	total: 51.6ms	remaining: 51.6ms
1:	learn: 0.4142747	total: 99.2ms	remaining: 0us
0:	learn: 0.4883075	total: 52.8ms	remaining: 52.8ms
1:	learn: 0.4394317	total: 101ms	remaining: 0us
0:	learn: 0.5673657	total: 52.8ms	remaining: 52.8ms
1:	learn: 0.5024726	total: 96.8ms	remaining: 0us
0:	learn: 0.4331312	total: 53.9ms	remaining: 53.9ms
1:	learn: 0.4081307	total: 106ms	remaining: 0us
0:	learn: 0.4858057	total: 89.4ms	remaining: 89.4ms
1:	learn: 0.4391219	total: 178ms	remaining: 0us
0:	learn: 0.5658406	total: 117ms	remaining: 117ms
1:	learn: 0.5005735	total: 190ms	remaining: 0us
0:	learn: 0.4453251	total: 105ms	remaining: 210ms
1:	learn: 0.4219567	total: 203ms	remaining: 101ms


In [ ]:
##Hyper-Parameters tuning for group

test_permutations = list(product(iterations_list,depth_list,learning_rate_list))

for iterations,depth,learning_rate in tqdm(test_permutations):
    user_splits_df=user_splits.copy()
    predictions,algo,user_splits_df = train_and_test_gb(x_train_group,y_train_group,categorical_features_indices_group,x_val_group
                                                        ,user_splits_df,iterations,depth,learning_rate,group=True)
    param_dict = {'iterations':iterations,'depth':depth,'learning_rate':learning_rate}
    user_splits_df, all_dict = evaluate_val_metrics_at_n_group(user_splits_df,'CatBoost_tag', n=10,model_params=param_dict)

  0%|          | 0/27 [00:00<?, ?it/s]

0:	learn: 0.4503813	total: 46.5ms	remaining: 46.5ms
1:	learn: 0.4142205	total: 90.7ms	remaining: 0us
0:	learn: 0.5015796	total: 47.1ms	remaining: 47.1ms
1:	learn: 0.4475400	total: 97.6ms	remaining: 0us
0:	learn: 0.5766648	total: 95.5ms	remaining: 95.5ms
1:	learn: 0.5115276	total: 183ms	remaining: 0us
0:	learn: 0.4385649	total: 106ms	remaining: 106ms
1:	learn: 0.4021537	total: 172ms	remaining: 0us
0:	learn: 0.4907174	total: 92.4ms	remaining: 92.4ms
1:	learn: 0.4438898	total: 181ms	remaining: 0us
0:	learn: 0.5692494	total: 53.1ms	remaining: 53.1ms
1:	learn: 0.5044032	total: 99.2ms	remaining: 0us
0:	learn: 0.4339531	total: 53.9ms	remaining: 53.9ms
1:	learn: 0.3968265	total: 104ms	remaining: 0us
0:	learn: 0.4869929	total: 53.9ms	remaining: 53.9ms
1:	learn: 0.4421572	total: 107ms	remaining: 0us
0:	learn: 0.5669448	total: 55.7ms	remaining: 55.7ms
1:	learn: 0.5021400	total: 105ms	remaining: 0us
0:	learn: 0.4503813	total: 46.6ms	remaining: 93.2ms
1:	learn: 0.4142205	total: 90.6ms	remaining: 45

In [ ]:
# Retrieve the best model parameters from the JSON file for the specified method
best_model_params_outfit = retrieve_best_model_params_from_file_outfit('CatBoost_tag', n=10)
best_model_params_group = retrieve_best_model_params_from_file_group('CatBoost_tag', n=10)

user_splits_df=user_splits.copy()
iterations_outfit=best_model_params_outfit['iterations']
depth_outfit=best_model_params_outfit['depth']
learning_rate_outfit=best_model_params_outfit['learning_rate']

iterations_group=best_model_params_group['iterations']
depth_group=best_model_params_group['depth']
learning_rate_group=best_model_params_group['learning_rate']

# Train the model with the best parameters
predictions,algo,user_splits_df = train_and_test_gb(x_join,y_join,categorical_features_indices_outfit,x_test,user_splits_df,
                                                    iterations_outfit, depth_outfit, learning_rate_outfit)
predictions,algo,user_splits_df = train_and_test_gb(x_join_group,y_join_group,categorical_features_indices_group,x_test_group,user_splits_df,
                                                    iterations_group, depth_group, learning_rate_group,group=True)
# Evaluate the model
user_splits_df, all_dict = evaluate_df_metrics_at_n(user_splits_df,'CatBoost_tag', n=10)

0:	learn: 0.4501656	total: 57.5ms	remaining: 57.5ms
1:	learn: 0.4258737	total: 109ms	remaining: 0us
0:	learn: 0.4573238	total: 51.7ms	remaining: 51.7ms
1:	learn: 0.4182363	total: 110ms	remaining: 0us


,0
id_hit_rate_at_10,0.0
id_precision_at_10,0.0
id_recall_at_10,0.0
id_f1_score_at_10,0.0
id_ndcg_at_10,0.0
group_hit_rate_at_10,0.0
group_precision_at_10,0.0
group_recall_at_10,0.0
group_f1_score_at_10,0.0
group_ndcg_at_10,0.0
